# To Do 

1. spacial drop out 
1. Time distibuted layers

### Done
1. Scaling  
1. shifting +1, +2 
1. Rolling  3 + 6

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

import tensorflow as tf
from tensorflow.keras import layers, models,callbacks,metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import roc_auc_score, auc

import gc

In [ ]:
BATCH_SIZE = 512
EPOCHS = 50
FOLDS = 10

DROP_SENSOR= False

SCALING = True
ADD_FEATURES = False

SHIFT_VALS = True

In [ ]:
train_original = pd.read_csv("../input/tabular-playground-series-apr-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-apr-2022/test.csv")
train_lables = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")
sub= pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv", index_col = 0)

In [ ]:
sensor_cols = [col for col in train_original.columns if "sensor" in col]

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 256
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

# Shift data

* Groupby sequence & subject  or just sequence 

In [ ]:
def shift_vals(df):
    for col in sensor_cols:
        df[f"{col}_shift1"] =df.groupby(["sequence","subject"])[col].shift(1).bfill()
        df[f"{col}_shift-1"] =df.groupby(["sequence","subject"])[col].shift(-1).ffill()
        
        df[f"{col}_rolling6"] = df.groupby(["sequence","subject"])[col].shift(1).rolling(6).mean().fillna(0)
        #df[f"{col}_rolling15"] = df.groupby(["sequence","subject"])[col]..shift(1).rolling(15).mean().fillna(0)
        df[f"{col}_rolling10"] = df.groupby(["sequence","subject"])[col].shift(1).rolling(10).mean().fillna(0)
        
        df[col + '_diff1'] = df[col] - df[f"{col}_shift1"]    
        df[col + '_diff-1'] = df[col] - df[f"{col}_shift-1"] 

    return df

if SHIFT_VALS:
    print("shifting values")
    shift_vals(train_original)
    shift_vals(test)

### Drop the sensor data

In [ ]:
if DROP_SENSOR:
    print("dropping raw sensor data")
    train = train.drop(sensor_cols,axis =1)
    test = test.drop(sensor_cols,axis =1)

In [ ]:
features_prior = [col for col in test.columns if col not in ["sequence","subject"]]

# Additional Features

In [ ]:
def add_features(df):
    new_df = pd.DataFrame()
    df_pivot = df.pivot(index = ["sequence","subject"], columns ="step", values = sensor_cols)
    
    for col in sensor_cols:
#         new_df[f"mean_{col}"] = df_pivot[col].mean(axis = 1)
#         new_df[f"median_{col}"] = df_pivot[col].median(axis = 1)
#         new_df[f"std_{col}"] = df_pivot[col].std(axis = 1)
#         new_df[f"variance_{col}"] = df_pivot[col].std(axis = 1)
#         new_df[f"max_{col}"] = df_pivot[col].max(axis = 1)
#         new_df[f"min_{col}"] = df_pivot[col].min(axis = 1)
#         new_df[f"max-min_{col}"] = df_pivot[col].max(axis = 1) - df.min(axis = 1)
        new_df[f"q50_{col}"] = df_pivot[col].quantile(q= 0.5, axis =1)
        new_df[f"q25_{col}"] = df_pivot[col].quantile(q= 0.25, axis =1) 
        new_df[f"q75_{col}"] = df_pivot[col].quantile(q= 0.75, axis =1)
        new_df[f"q95_{col}"] = df_pivot[col].quantile(q= 0.95, axis =1)
        new_df[f"q99_{col}"] = df_pivot[col].quantile(q= 0.99, axis =1)
        new_df[f"skew_{col}"] = df_pivot[col].skew( axis =1)
    
    df = df.merge(new_df, how = "left", on = "sequence")   
    return df

if ADD_FEATURES:
    print("Adding Features")
    train = add_features(train_original)
    test = add_features(test)

else :
    train = train_original.copy()

from https://www.kaggle.com/code/hasanbasriakcay/tpsapr22-fe-pseudo-labels-bi-lstm/notebook

In [ ]:
def create_new_features(df):
    df['sensor_02_num'] = df['sensor_02'] > -15
    df['sensor_02_num'] = df['sensor_02_num'].astype(int)
    df['sensor_sum1'] = (df['sensor_00'] + df['sensor_09'] + df['sensor_06'] + df['sensor_01'])
    df['sensor_sum2'] = (df['sensor_01'] + df['sensor_11'] + df['sensor_09'] + df['sensor_06'] + df['sensor_00'])
    df['sensor_sum3'] = (df['sensor_03'] + df['sensor_11'] + df['sensor_07'])
    df['sensor_sum4'] = (df['sensor_04'] + df['sensor_10'])
    
    return df 

#create_new_features (train)
#create_new_features (test)

In [ ]:
print([col for col in train.columns])

# Scaling and Reshaping 

**NOTE**: Scaing before splitting causes leakage however Im unsure how to scale after reshaping the data 

We also need to reshape the data for LSTM in the format: 
* samples 
* timesteps 
* features 

i.e. reshape.(samples, timesteps, features) 

In [ ]:
train= train.drop(["sequence","subject"],axis =1).set_index("step")
test= test.drop(["sequence","subject"],axis =1).set_index("step")
train

In [ ]:
FEATURES = len(train.columns)
add_features = [col for col in train.columns if col not in features_prior]

In [ ]:
if SCALING:
    print("scaling")
    scaler= StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)

In [ ]:
train = train.reshape( int(len(train)/60), 60, FEATURES)
test = test.reshape( int(len(test)/60), 60, FEATURES)

# Split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, train_lables["state"], test_size=0.33, shuffle = False)

# Build and Train

Credit for this model goes to ROLAND ABEL
https://www.kaggle.com/code/ted0071/tps-apr-2022-keras-lstm-model#LSTM-Model

In [ ]:
print("Features Length:", FEATURES)

In [ ]:
def build_model():
    
    x_input = layers.Input(shape=(60, FEATURES))
    b = layers.BatchNormalization()(x_input)
    x1 = layers.Bidirectional(layers.LSTM(units=768, return_sequences=True,dropout=0.15, recurrent_dropout=0.15))(b)  
    x2 = layers.Bidirectional(layers.LSTM(units=512, return_sequences=True,dropout=0.15, recurrent_dropout=0.15))(x1)
    z1 = layers.Bidirectional(layers.GRU(units=512, return_sequences=True))(x1)
    c = layers.Concatenate(axis=2)([x2, z1])
    
    x31 = layers.Bidirectional(layers.LSTM(units=128, return_sequences=True))(c)
    x32 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(c)
    l3 = layers.Concatenate(axis=2)([x31, x32])
    
    x4 = layers.GlobalMaxPooling1D()(l3) 
#     avg_pool = layers.GlobalAveragePooling1D()(l3)
#     max_pool = layers.GlobalMaxPooling1D()(l3)
#     c2 = layers.concatenate([avg_pool, max_pool])
    
    #x5 = layers.Dense(units=128, activation='selu')(x4)
    x_output = layers.Dense(1, activation='sigmoid')(x4)
    
    model = models.Model(inputs=x_input, outputs=x_output, name='lstm_model')
    model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=[metrics.AUC(name = 'auc')])
    
    return model

model = build_model()

In [ ]:
model.summary()

In [ ]:
with tpu_strategy.scope():
    model = build_model()
    model.fit(X_train ,
              y_train,
              epochs= EPOCHS, 
              callbacks= [callbacks.EarlyStopping(patience=10,monitor='val_auc', mode = "max") ,
                          callbacks.ReduceLROnPlateau(monitor="val_auc", mode = "max",patience = 4, factor= 0.1)],
              validation_data=(X_test,
                               y_test), 
              batch_size = BATCH_SIZE
             )

In [ ]:
history = pd.DataFrame(model.history.history)
history

In [ ]:
val_preds = model.predict(X_test, batch_size = BATCH_SIZE)
train_preds = model.predict(X_train, batch_size = BATCH_SIZE)
train_preds

In [ ]:
print("Validation AUC:" , roc_auc_score(y_test, val_preds ))
print("Intrinsic AUC:", roc_auc_score(y_train, train_preds ))

In [ ]:
history[["loss","val_loss"]].plot(figsize = (20,8))
plt.title("Training vs Validation Loss")
plt.show()

In [ ]:
history[["auc","val_auc"]].plot(figsize = (20,8))
plt.title("Training vs Validation AUC")
plt.show()

# Cross Validation

In [ ]:
del model
del X_train
del y_train
del X_test
del y_test
del val_preds
del train_preds
del history

gc.collect()

In [ ]:
X = train
y = train_lables["state"]

In [ ]:
kfold = GroupKFold(n_splits = FOLDS)

In [ ]:
auc_cv = []
preds = []

for fold, (train_idx, val_idx) in enumerate (kfold.split(X,y, groups =train_original.sequence.unique())):

    print("\n","#"*10, f"Fold {fold+1}","#"*10)
    X_train, X_test = X[train_idx] , X[val_idx]
    y_train , y_test = y[train_idx], y[val_idx]
    
    with tpu_strategy.scope():
        model = build_model()

        model.compile(optimizer="adam", loss="binary_crossentropy", metrics="AUC")
        model.fit(X_train,y_train,epochs= EPOCHS, 
                  callbacks= [callbacks.EarlyStopping(patience=10,monitor='val_auc', mode = "max") ,
                              callbacks.ReduceLROnPlateau(monitor="val_auc",mode = "max",patience = 4, factor= 0.1 ,restore_best_weights=True)],
                  validation_data=(X_test,y_test), batch_size = BATCH_SIZE)

        auc = roc_auc_score(y_test, model.predict(X_test,batch_size = BATCH_SIZE))
        print("\n Validation AUC:" , auc)

        auc_cv.append(auc)
        preds.append(model.predict(test,batch_size = BATCH_SIZE).squeeze())
    
    del X_train
    del X_test
    del y_train
    del y_test
    del model
    
    gc.collect()

print("FINAL AUC: ", np.mean(auc_cv))

# Submission

In [ ]:
final_preds = np.sum(preds,axis =0)/FOLDS
sub["state"] = final_preds
sub.to_csv("submission.csv")
sub

In [ ]:
plt.figure(figsize = (20,8))
sns.histplot(sub["state"])
plt.show()